In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
labels=pd.read_csv('../input/data-science-london-scikit-learn/trainLabels.csv',header=None)

In [ ]:
labels.head()
labels.info()

In [ ]:
train=pd.read_csv('../input/data-science-london-scikit-learn/train.csv',header=None)
train.head()

In [ ]:
test=pd.read_csv('../input/data-science-london-scikit-learn/test.csv',header=None)

In [ ]:
test.info()

In [ ]:
train.info()

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train,labels,test_size=0.3,random_state=99)

In [ ]:
print(labels)

In [ ]:
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV, cross_val_score
x_all = np.r_[train,test]
print('x_all shape :',x_all.shape)

# USING THE GAUSSIAN MIXTURE MODEL 
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type)
        gmm.fit(x_all)
        bic.append(gmm.aic(x_all))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
            
best_gmm.fit(x_all)
gmm_train = best_gmm.predict_proba(train)
gmm_test = best_gmm.predict_proba(test)

In [ ]:
##logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lg=LogisticRegression()
lg.fit(X_train,y_train)
preds=lg.predict(X_test)
print(y_train)
print("accuracy score:",accuracy_score(y_test,preds))

In [ ]:
##RFC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
rfc_b = RFC(random_state=99)
param_grid = { 
    'n_estimators': [10,50,100,200, 500],

    'max_depth' : [3,10,20,40],

}
CV_rfc = GridSearchCV(estimator=rfc_b, param_grid=param_grid, cv= 10,scoring='accuracy',n_jobs=-1)
CV_rfc.fit(gmm_train, labels.values.ravel())
rfc_best = CV_rfc.best_estimator_

#rfc_b.fit(X_train,y_train)
#y_pred = rfc_b.predict(X_test)
#print('accuracy score:', accuracy_score(y_test,y_pred))

In [ ]:
CV_rfc.best_score_

In [ ]:
rfc1=rfc_best
rfc1.fit(gmm_train, labels.values.ravel())
y_pred = rfc1.predict(gmm_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))

In [ ]:
##SVC
from sklearn import svm
clf = svm.SVC(kernel='poly',degree=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))

In [ ]:
##SVC
from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))

In [ ]:
##SVC
from sklearn import svm
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('accuracy score:',accuracy_score(y_test,y_pred))


In [ ]:
rfc_best.fit(gmm_train,labels.values.ravel())
pred  = rfc_best.predict(gmm_test)
rfc_best_pred = pd.DataFrame(pred)

rfc_best_pred.index += 1

rfc_best_pred.columns = ['Solution']
rfc_best_pred['Id'] = np.arange(1,rfc_best_pred.shape[0]+1)
rfc_best_pred = rfc_best_pred[['Id', 'Solution']]


rfc_best_pred.to_csv('Submission.csv',index=False)